# Présentation du code Open-Source FEniCSx

---

## Contenu du cours
1. Structure d'un code éléments finis
2. Résolution du devoir 3
    * Utilisation du code FEniCSx à haut niveau
    * Utilisation du code FEniCSx et PETSC
3. Présentation de l'adjoint
    * Problème couplé
    * Définition du problème adjoint


---

# Structure d'un code éléments finis

<img src="diagram_fenicsx.svg" alt="Structure code Fenicsx" width="800"/>

---

# Problème 2a) du devoir 3

On reprend le problème du devoir 3, où l'on étudie la déflection d'une corde. Cette déflection est modélisée par l'équation de la caténaire  linéarisée:

\begin{align}
    -k u''(x) &= -\rho g \qquad &&\forall x\in \Omega = (0,L),\\
    u(0) &= 0, &&\\
    k u'(L) &= t, &&
\end{align}

où $t\in\mathbb{R}$. On considère l'espace $V = \{v\in H^1(0,L)\,|\, v(0)=0\}$. La forme faible de ce problème est

\begin{align}
    \int_0^L k u'(x) v'(x)\, dx = \int_0^L f(x) v(x)\, dx + t v(L) \qquad \forall v \in V
\end{align}

Le maillage employé pour ce problème est le suivant

<img src="./mesh_corde.svg" alt="Structure code Fenicsx" width="800"/>

## Importation des modules nécessaires

In [3]:
# %pip install ipyvtklink

import numpy as np
import pyvista

import ufl # Package pour manipuler les formes bilinéaires et linéaires
from dolfinx import fem, plot # Package pour l'assemblage et pour l'affichage
from dolfinx.io import XDMFFile, gmshio # Package avec des fonctions pour l'input/output de données

from mpi4py import MPI
from petsc4py import PETSc


## Définition du maillage

2 approches peuvent être employées afin de définir un maillage:
1. Utiliser les maillages simples de FEniCSx,
2. Importer ou créer un maillage avec GMSH.

On adoptera la seconde approche puisqu'elle se généralise mieux et permet de le traitement simplifiés des conditions frontières.

In [4]:
# (domain, cell_tags, facet_tags) = gmshio.read_from_msh("./data/mesh/verification/unit_square.msh", MPI.COMM_WORLD, gdim=2)
(domain, cell_tags, facet_tags) = gmshio.read_from_msh("corde.msh", MPI.COMM_WORLD, gdim=1)

x = ufl.SpatialCoordinate(domain) # Coordonnées pour la définition des formes
ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tags) # ds(i) contient l'information sur la i-ème frontière
nMesh = ufl.FacetNormal(domain) # Vecteur normal au domaine

Info    : Reading 'corde.msh'...
Info    : 9 entities
Info    : 11 nodes
Info    : 15 elements
Info    : Done reading 'corde.msh'


## Définition des paramètres du problème

In [5]:
L = 1
t = 5
k = 10
rho_g = 10

## Définition de l'espace éléments finis $V$

In [36]:
degre = 2
V = fem.FunctionSpace(domain, ("Lagrange", degre))
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

## Définition de la forme bilinéaire

In [37]:
bilinear = k * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx

## Définition des conditions frontières et de la forme linéaire

On définit tout d'abord les conditions de Dirichlet en créant une fonction appartenant à $V$. Cette fonction interpolera la ou les conditions de Dirichlet (cette fonction peut être interprétée comme le relèvement).

In [38]:
def fct_Dirichlet2(x):
    return 0.0* x[0] # x[0] de renvoyer un array de bonne structure, car x est un array de dim x nb_points 

u_D = fem.Function(V)
u_D.interpolate(fct_Dirichlet2)
facets = facet_tags.find(1) # Retourne les entités géométrique sur la frontière 1 (le noeud à gauche pour ce problème)
dofs = fem.locate_dofs_topological(V, 0, facets) # Retourne les dofs sur la frontière 1 (le noeud à gauche pour ce problème)
bcs = fem.dirichletbc(u_D, dofs)

On définit la forme linéaire en incorporant la condition de Neumann

In [39]:
f = -rho_g
linear_a = f*v*ufl.dx + t*v*ds(5)

## Assemblage et résolution du système linéaire

In [40]:
problem_a = fem.petsc.LinearProblem(bilinear, linear_a, bcs=[bcs], petsc_options={"ksp_type": "preonly", "pc_type": "lu"}) # Factorisation LU
uh_a = problem_a.solve()

In [58]:
pyvista.set_jupyter_backend("pythreejs")

cells, types, coord = plot.create_vtk_mesh(V)
pdata = pyvista.PolyData(coord)
pdata.lines = cells
pdata.point_data["u"] = uh_a.x.array.real
pdata.set_active_scalars("u")
plotter = pyvista.Plotter()
plotter.add_mesh(pdata, style='points',show_edges=True,color='black')
plotter.add_mesh(pdata, show_edges=True,color='black')
warped = pdata.warp_by_scalar()
plotter.add_mesh(warped)
plotter.view_xz()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    pyvista.start_xvfb()
    figure = plotter.screenshot("sol_a.png")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

## Calcul de l'erreur

On calcule l'erreur en interpolant la solution exacte sur un espace éléments finis enrichis. La solution exacte est

\begin{align}
    u(x) = -\frac{x(1-x)}{2}, \qquad x\in [0,1]
\end{align}

On prendra ici des éléments de Lagrange d'un degré plus élevé. 

In [42]:
uex_a_fct = lambda x: -x[0] * (1-x[0])/2

Vplus = fem.FunctionSpace(domain, ("Lagrange", degre + 1))

uex = fem.Function(Vplus)
uex.interpolate(uex_a_fct)

error = uh_a - uex

L2_form = fem.form(ufl.inner(error, error) * ufl.dx)
L2_local = fem.assemble_scalar(L2_form)
L2 = np.sqrt(domain.comm.allreduce(L2_local, op=MPI.SUM))

H1_semi_form = fem.form(ufl.inner(ufl.grad(error), ufl.grad(error)) * ufl.dx)
H1_semi_local = fem.assemble_scalar(H1_semi_form)
H1 = np.sqrt(L2**2 + domain.comm.allreduce(H1_semi_local, op=MPI.SUM))

In [43]:
print(f"Erreur L2: {L2}")
print(f"Erreur H1: {H1}")

Erreur L2: 2.649799250827198e-15
Erreur H1: 5.2563520003802096e-15


On peut aussi calculer l'erreur en chaque point du maillage. Pour ce faire, il faut interpoler la solution exacte sur le même espace fonctionelle $V_1$

In [59]:
uex_interV = fem.Function(V)
uex_interV.interpolate(uex_a_fct)

cells, types, coord = plot.create_vtk_mesh(V)
pdata = pyvista.PolyData(coord)
pdata.lines = cells
pdata.point_data["u"] = np.abs(uh_a.x.array.real[:] - uex_interV.x.array.real[:])
pdata.set_active_scalars("u")
plotter = pyvista.Plotter()
plotter.add_mesh(pdata, style='points',show_edges=True,color='black')
plotter.add_mesh(pdata,show_edges=True,color='black')

warped = pdata.warp_by_scalar()
plotter.add_mesh(warped, style='points',show_edges=True)
plotter.view_xz()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    pyvista.start_xvfb()
    figure = plotter.screenshot("erreur_node.png")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

---

# Problème 2b) du devoir 3

Pour ce problème, on change le terme source pour

\begin{align}
    f(x) = -\rho g - f_0 \delta(x-x_0),
\end{align}

où $f_0=20$ et $x_0=0.4$. Ce chargement correspond donc au poids de la corde additionné d'une force ponctuelle de valeur $f_0$ appliquée en $x=x_0$.

Ce problème est un peu plus compliqué à résoudre, mais cela nous permet d'utiliser FEniCSx à un niveau plus bas i.e. en assemblant nous même le système linéaire à résoudre.

## Assemblage de la matrice $A$

In [45]:
bilinear_form = fem.form(bilinear)
linear_form = fem.form(linear_a)

A = fem.petsc.assemble_matrix(bilinear_form, bcs=[bcs])
A.assemble()

## Assemblage du terme de droite $B$

Le terme de droite est un peu plus compliqué à assembler. On doit tout d'abord appliquer le "lifting" des conditions de Dirichlet et ensuite appliquer ces mêmes valeurs de Dirichlet.

In [46]:
b = fem.petsc.create_vector(linear_form)

# with b.localForm() as loc_b:
#         loc_b.set(0)
fem.petsc.assemble_vector(b, linear_form)

# Apply Dirichlet boundary condition to the vector
fem.petsc.apply_lifting(b, [bilinear_form], [[bcs]])
b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
fem.petsc.set_bc(b, [bcs])

Finalement, il faut ajouter le terme de Dirac au vecteur. On calcule le terme à ajouter

\begin{align}
    -\int_0^L f_0 \delta(x - x_0)v(x)\, dx = -f_0(x_0)v(x_0)
\end{align}

Puisque l'on emploie des éléments de Lagrange, il faut identifier le degré de liberté qui ne vaut pas 0 en $x=x_0$.  

In [47]:
facets_x0 = facet_tags.find(3) # Retourne les entités géométrique sur la frontière 3 (le noeud exactement à x=0.4)
dofs_x0 = fem.locate_dofs_topological(V, 0, facets_x0)

f_0 = -20

b.setValue(dofs_x0,f_0,addv=PETSc.InsertMode.ADD_VALUES)
b.assemblyBegin()
b.assemblyEnd()

## Résolution du système linéaire

On peut maintenant faire appel à la libraire PETSC qui permet de résoudre des systèmes linéaires en parallèle. On peut ainsi avoir beacoup de contrôle sur quel type de solveur et de préconditionneur sont employés, les tolérances et le nombre d'itérations maximal.

In [48]:
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType("preonly")
solver.getPC().setType("lu")

uh_b = fem.Function(V)

# Solve linear problem
solver.solve(b, uh_b.vector)
uh_b.x.scatter_forward()

In [60]:
pyvista.set_jupyter_backend("pythreejs")

cells, types, coord = plot.create_vtk_mesh(V)
pdata = pyvista.PolyData(coord)
pdata.lines = cells
pdata.point_data["u"] = uh_b.x.array.real
pdata.set_active_scalars("u")
plotter = pyvista.Plotter()
plotter.add_mesh(pdata, style='points',show_edges=True,color='black')
plotter.add_mesh(pdata, show_edges=True,color='black')

warped = pdata.warp_by_scalar()
plotter.add_mesh(warped)
plotter.view_xz()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    pyvista.start_xvfb()
    figure = plotter.screenshot("sol_b.png")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

## Calcul de l'erreur

La solution exacte de ce problème avec ce chargement Dirac additionel est

\begin{align}
    u(x) = -\frac{x(1-x)}{2} - \begin{cases} 2x & x\in [0,0.4] \\
     4/5 & x\in[0.4,1]\end{cases}
\end{align}

On peut encore une fois représenter exactement cette solution avec un espace discret composé de polynomes de degré 2.

In [50]:
def uex_b_fct(x):
    f = np.zeros(np.size(x,1))
    f = -x[0]*(1-x[0])/2 - 2*x[0] * (x[0] <=0.4) + -4/5 * (x[0]>0.4)
    return f

uex = fem.Function(Vplus)
uex.interpolate(uex_b_fct)

error = uh_b - uex

L2_form = fem.form(ufl.inner(error, error) * ufl.dx)
L2_local = fem.assemble_scalar(L2_form)
L2 = np.sqrt(domain.comm.allreduce(L2_local, op=MPI.SUM))

H1_semi_form = fem.form(ufl.inner(ufl.grad(error), ufl.grad(error)) * ufl.dx)
H1_semi_local = fem.assemble_scalar(H1_semi_form)
H1 = np.sqrt(L2**2 + domain.comm.allreduce(H1_semi_local, op=MPI.SUM))

In [51]:
print(f"Erreur L2: {L2}")
print(f"Erreur H1: {H1}")

Erreur L2: 2.3398143127120012e-14
Erreur H1: 4.483872884727257e-14


---

# Problème 2c) du devoir 3

Pour ce problème, on change le terme source pour

\begin{align}
    f(x) = -\rho g - f_0 x^2\big[H(x-x_1) - H(x-x_2)\big],
\end{align}

où $f_0=500$ et $x_1=0.2$ et $x_2=0.6$. Ce chargement correspond donc au poids de la corde additionné d'une force ponctuelle de valeur $f_0$ appliquée en $x=x_0$.


In [52]:
f_0 = 500
l_charge = ufl.conditional(ufl.And(ufl.ge(x[0],0.2),ufl.le(x[0],0.6)),-f_0 * x[0]**2,0)

linear_c = linear_a + l_charge*v*ufl.dx

In [53]:
problem_c = fem.petsc.LinearProblem(bilinear, linear_c, bcs=[bcs], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh_c = problem_c.solve()

In [61]:
cells, types, coord = plot.create_vtk_mesh(V)
pdata = pyvista.PolyData(coord)
pdata.lines = cells
pdata.point_data["u"] = uh_c.x.array.real
pdata.set_active_scalars("u")
plotter = pyvista.Plotter()
plotter.add_mesh(pdata, style='points',show_edges=True,color='black')
plotter.add_mesh(pdata, show_edges=True,color='black')

warped = pdata.warp_by_scalar()
plotter.add_mesh(warped)
plotter.view_xz()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    pyvista.start_xvfb()
    figure = plotter.screenshot("sol_c.png")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

## Calcul de l'erreur

Avec ce nouveau chargement, la solution exacte est 

\begin{align}
    u(x) = -\frac{x(1-x)}{2} - \begin{cases} \frac{52}{15}x & x\in [0,0.2] \\
     -\frac{25}{6}x^4 + \frac{18}{5}x - \frac{1}{50} & x\in[0.2,0.6] \\
     \frac{8}{5} & x\in[0.6,1]\end{cases}
\end{align}

In [55]:
def uex_c_fct(x):
    f = np.zeros(np.size(x,1))
    f = -x[0]*(1-x[0])/2
    f = f - 52/15*x[0] * (x[0] <=0.2)
    f = f - (-25/6*x[0]**4 + 18/5*x[0] - 1/50) * ((x[0] > 0.2) & (x[0] < 0.6))
    f = f - 8/5 * (x[0]>=0.6)
    return f

uex = fem.Function(Vplus)
uex.interpolate(uex_c_fct)

error = uh_c - uex

L2_form = fem.form(ufl.inner(error, error) * ufl.dx)
L2_local = fem.assemble_scalar(L2_form)
L2 = np.sqrt(domain.comm.allreduce(L2_local, op=MPI.SUM))

H1_semi_form = fem.form(ufl.inner(ufl.grad(error), ufl.grad(error)) * ufl.dx)
H1_semi_local = fem.assemble_scalar(H1_semi_form)
H1 = np.sqrt(L2**2 + domain.comm.allreduce(H1_semi_local, op=MPI.SUM))

In [56]:
print(f"Erreur L2: {L2}")
print(f"Erreur H1: {H1}")

Erreur L2: 0.0001510544945280732
Erreur H1: 0.009790615445087317


In [57]:
if degre == 2:
    print("Interpolation de l'erreur pour 5 éléments quadratiques")
    print(f"Erreur L2: {8*L2}")
    print(f"Erreur H1: {4*H1}")

Interpolation de l'erreur pour 5 éléments quadratiques
Erreur L2: 0.0012084359562245855
Erreur H1: 0.03916246178034927
